In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the pre-trained model
model = load_model("/content/drive/MyDrive/skin diseases/skin disease model.h5", compile=False)

# Load the class names (labels)
class_names = open("/content/drive/MyDrive/skin diseases/skin disease labels.txt", "r").readlines()

# Define the image size and shape for the model
image_size = (224, 224)
data = np.ndarray(shape=(1, *image_size, 3), dtype=np.float32)

# Input image path
image_path = input("Insert image path: ")

# Preprocess the image
image = Image.open(image_path).convert("RGB")
image = ImageOps.fit(image, image_size, Image.Resampling.LANCZOS)
image_array = np.asarray(image)
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
data[0] = normalized_image_array

# Make a prediction
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index].strip()  # Remove newline characters
confidence_score = prediction[0][index]

# Print the prediction and confidence score
print(f"Class: {class_name[2:]}")  # Skip the first two characters (likely an index or bullet)
print(f"Confidence Score: {confidence_score}")

# Define the URLs for the conditions
urls = {
    'Acne': 'https://www.mayoclinic.org/diseases-conditions/acne/symptoms-causes/syc-20368047',
    'Cellulitis': 'https://www.mayoclinic.org/diseases-conditions/cellulitis/symptoms-causes/syc-20370762',
    'Dermatitis': 'https://www.mayoclinic.org/diseases-conditions/dermatitis-eczema/symptoms-causes/syc-20352380',
    'Eczema': 'https://www.mayoclinic.org/diseases-conditions/atopic-dermatitis-eczema/symptoms-causes/syc-20353273',
    'Ivy': 'https://www.mayoclinic.org/diseases-conditions/poison-ivy/symptoms-causes/syc-20376485',
    'Psoriasis': 'https://www.mayoclinic.org/diseases-conditions/psoriasis/symptoms-causes/syc-20355840',
    'Scabies': 'https://www.mayoclinic.org/diseases-conditions/scabies/symptoms-causes/syc-20377378'
}

# Select the appropriate URL based on the predicted class
url = urls.get(class_name.split(' ')[-1], 'https://www.mayoclinic.org/diseases-conditions/common-warts/symptoms-causes/syc-20371125')
print(url)

# Scrape the content from the selected URL
r = requests.get(url)
if r.status_code == 200:
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')

    # Define specific CSS selector for Cellulitis
    if 'Cellulitis' in class_name:
        target_div = soup.select_one('#__next > div > div > div:nth-child(4) > div.cmp-yspace.cmp-yspace-top-xsm.cmp-yspace-bottom-xsm > div > div > div:nth-child(1) > article > div > div:nth-child(2) > div > div')
        if target_div:
            content = target_div.get_text(strip=True)
        else:
            content = "No content found using the specified CSS selector."
    else:
        # Function to extract text based on section header
        def extract_section(header):
            section = soup.find(['h2', 'h3', 'h4'], string=header)
            if section:
                content = []
                for sibling in section.find_next_siblings():
                    if sibling.name in ['h2', 'h3', 'h4']:
                        break
                    content.append(sibling.get_text(separator=' ', strip=True))
                return ' '.join(content)
            return "Section not found."

        # Summarization function using transformers
        summarizer = pipeline("summarization")

        def summarize_text(text, max_length=150, min_length=40):
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
            return summary[0]['summary_text']

        # Extract and summarize specific sections
        overview = extract_section("Overview")
        symptoms = extract_section("Symptoms")
        causes = extract_section("Causes")
        risk_factors = extract_section("Risk factors")
        if risk_factors == "Section not found.":
            risk_factors = extract_section("Risk Factors")

        # Summarize the extracted sections
        if overview != "Section not found.":
            overview = summarize_text(overview)
        if symptoms != "Section not found.":
            symptoms = summarize_text(symptoms)
        if causes != "Section not found.":
            causes = summarize_text(causes)
        if risk_factors != "Section not found.":
            risk_factors = summarize_text(risk_factors)

        # Compile content
        content = []
        if overview != "Section not found.":
            content.append(f"Overview: {overview}")
        if symptoms != "Section not found.":
            content.append(f"Symptoms: {symptoms}")
        if causes != "Section not found.":
            content.append(f"Causes: {causes}")
        if risk_factors != "Section not found.":
            content.append(f"Risk Factors: {risk_factors}")

        content = '\n'.join(content) if content else "No content found."

    # Print the extracted or summarized content
    print(content)
else:
    print(f"Failed to retrieve the webpage. Status code: {r.status_code}")


ModuleNotFoundError: No module named 'transformers'